In [1]:
import numpy as np
import torch.nn as nn
import matplotlib
import random
import torch

from itertools import islice
from sklearn.utils import shuffle
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset

from models.Encoder import *
from models.Decoder import *
from models.utils import *
from utils import *

# set fixed random seed to reproduce results
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

%matplotlib inline

CPU used
CPU used


In [2]:
# define experiment
exp='/exp_2'

In [3]:
device = "cpu" #("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds_train, act_vocab, w2i_acts, i2w_acts, acts_train = load_dataset(exp=exp, split='/train')
_, _, _, cmds_test, _, _, _, acts_test = load_dataset(exp=exp, split='/test')

In [5]:
## create input and output language pairs ##

# training
train_cmd_act_pairs = create_pairs(cmds_train, acts_train)
print("Number of train source-target pairs: {}".format(len(train_cmd_act_pairs)))

# testing
test_cmd_act_pairs = create_pairs(cmds_test, acts_test)
print("Number of test source-target pairs: {}".format(len(test_cmd_act_pairs)))

Number of train source-target pairs: 16990
Number of test source-target pairs: 3920


In [6]:
# show random train command-action pair
random_pair = random.choice(train_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['jump', 'opposite', 'left', 'after', 'run', 'thrice']
Action: ['I_RUN', 'I_RUN', 'I_RUN', 'I_TURN_LEFT', 'I_TURN_LEFT', 'I_JUMP']


In [7]:
# show random test command-action pair
random_pair = random.choice(test_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['run', 'around', 'right', 'twice', 'and', 'look', 'around', 'right']
Action: ['I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_LOOK', 'I_TURN_RIGHT', 'I_LOOK', 'I_TURN_RIGHT', 'I_LOOK', 'I_TURN_RIGHT', 'I_LOOK']


In [8]:
cmd_act_pair = pairs2idx(random_pair, w2i_cmds, w2i_acts)
print("Command sequence: {}".format(cmd_act_pair[0]))
print("Action sequence: {}".format(cmd_act_pair[1]))

Command sequence: tensor([15, 11,  4,  6, 10, 14, 11,  4,  2])
Action sequence: tensor([1, 4, 9, 4, 9, 4, 9, 4, 9, 4, 9, 4, 9, 4, 9, 4, 9, 4, 8, 4, 8, 4, 8, 4,
        8, 2])


In [9]:
## Hyperparameters for training ##

# source language (i.e., commands) vocabulary size |V_source|
in_size = len(w2i_cmds)

# target language (i.e., actions) vocabulary size |V_target|
out_size = len(w2i_acts)

# size of word embeddings
emb_size = 20

# size of hidden units
hidden_size = 100

# number of layers
n_layers = 1

# number of epochs
n_epochs = 1

In [10]:
## Instantiate models ##

encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers)
decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers)

C:\Users\lukas\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [11]:
# move models to GPU, if GPU is available (for faster computation)
encoder.to(device)

EncoderLSTM(
  (embedding): Embedding(17, 20)
  (lstm): LSTM(20, 100, dropout=0.5)
)

In [12]:
decoder.to(device)

DecoderLSTM(
  (embedding): Embedding(10, 20)
  (lstm): LSTM(20, 100, dropout=0.5)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)

## Training

In [14]:
train_losses, train_accs, encoder, decoder = train(train_cmd_act_pairs, 
                                                   w2i_cmds, w2i_acts, 
                                                   i2w_cmds, i2w_acts,
                                                   encoder, decoder, 
                                                   n_epochs, 
                                                   detailed_analysis=True,
                                                   phrase_based_loss=True)

Epoch:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]

Loss: 0.2125701904296875
Acc: 0.000999000999000999

Command: run around right and jump opposite left twice <EOS>
True action: I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_LEFT I_TURN_LEFT I_JUMP I_TURN_LEFT I_TURN_LEFT I_JUMP <EOS>
Pred action: I_TURN_LEFT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT <EOS> <EOS> <EOS> <EOS>

True sent length: 15
Pred sent length: 15

Loss: 0.5285149680243598
Acc: 0.00399800099950025

Command: run opposite left twice and walk <EOS>
True action: I_TURN_LEFT I_TURN_LEFT I_RUN I_TURN_LEFT I_TURN_LEFT I_RUN I_WALK <EOS>
Pred action: I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT <EOS> <EOS> <EOS> <EOS>

True sent length: 8
Pred sent length: 8

Loss: 0.2473350895775689
Acc: 0.007664111962679107

Command: run thrice and jump right twice <EOS>
True action: I_RUN I_RUN I_RUN I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP <EOS>
Pred action: I_RUN I_

Epoch: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [09:18<00:00, 558.64s/it]


## Testing

In [ ]:
test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)